### Spark-NLP

#### The Data

The UCI ML News Aggregator dataset contains headlines and categories for over 400k news articles. Task is to build classification model that classifies news headline into pre-defined categories.
The data can be downloaded from https://www.kaggle.com/uciml/news-aggregator-dataset

#### Prerequisites:

Basics understand of Python

https://www.kaggle.com/learn/python

Natural Language Processing  
https://www.analyticsvidhya.com/blog/2017/01/ultimate-guide-to-understand-implement-natural-language-processing-codes-in-python/

Apache Spark

https://docs.databricks.com/spark/latest/gentle-introduction/gentle-intro.html

https://docs.databricks.com/spark/latest/gentle-introduction/gentle-intro.html#gentle-introduction-to-apache-spark

https://docs.databricks.com/spark/latest/gentle-introduction/for-data-scientists.html

#### Importing the necessary libraries

In [5]:
from pyspark.sql import SparkSession
from pyspark.ml import Pipeline 
from pyspark.ml.feature import CountVectorizer,StringIndexer, RegexTokenizer,StopWordsRemover
from pyspark.sql.functions import col, udf,regexp_replace,isnull
from pyspark.sql.types import StringType,IntegerType
from pyspark.ml.classification import NaiveBayes
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

#### Data Ingestion and Extraction

In this example, I am importing dataset from S3(https://docs.databricks.com/spark/latest/data-sources/aws/amazon-s3.html).

You can can directly upload dataset to databricks cloud(https://docs.databricks.com/user-guide/importing-data.html#import-data)

In [2]:
file_path = "nlp/train.csv"

In [8]:
from pyspark.sql.types import *
import pandas as pd
pd_df = pd.read_csv(file_path)
sp_df = spark.createDataFrame(pd_df, schema=schema)

NameError: name 'spark' is not defined

In [3]:
news_data = spark.read.csv(s3_bucket_path,header = 'True',inferSchema='True')

NameError: name 'spark' is not defined

In [9]:
news_data.show()

+---+--------------------+--------------------+--------------------+--------+--------------------+--------------------+-------------+
 ID| TITLE| URL| PUBLISHER|CATEGORY| STORY| HOSTNAME| TIMESTAMP|
+---+--------------------+--------------------+--------------------+--------+--------------------+--------------------+-------------+
 1|Fed official says...|http://www.latime...| Los Angeles Times| b|ddUyU0VZz0BRneMio...| www.latimes.com|1394470370698|
 2|Fed's Charles Plo...|http://www.livemi...| Livemint| b|ddUyU0VZz0BRneMio...| www.livemint.com|1394470371207|
 3|US open: Stocks f...|http://www.ifamag...| IFA Magazine| b|ddUyU0VZz0BRneMio...| www.ifamagazine.com|1394470371550|
 4|Fed risks falling...|http://www.ifamag...| IFA Magazine| b|ddUyU0VZz0BRneMio...| www.ifamagazine.com|1394470371793|
 5|Fed's Plosser: Na...|http://www.moneyn...| Moneynews| b|ddUyU0VZz0BRneMio...| www.moneynews.com|1394470372027|
 6|Plosser: Fed May ...|http://www.nasdaq...| NASDAQ| b|ddUyU0VZz0BRneMio...| www.nasdaq.com|1394470372212|
 7|Fed's Plosser: Ta...|http://www.market...| MarketWatch| b|ddUyU0VZz0BRneMio...| www.marketwatch.com|1394470372405|
 8|Fed's Plosser exp...|http://www.fxstre...| FXstreet.com| b|ddUyU0VZz0BRneMio...| www.fxstreet.com|1394470372615|
 9|US jobs growth la...|http://economicti...| Economic Times| b|ddUyU0VZz0BRneMio...|economictimes.ind...|1394470372792|
 10|ECB unlikely to e...|http://www.iii.co...|Interactive Investor| b|dPhGU51DcrolUIMxb...| www.iii.co.uk|1394470501265|
 11|ECB unlikely to e...|http://in.reuters...| Reuters India| b|dPhGU51DcrolUIMxb...| in.reuters.com|1394470501410|
 12|EU's half-baked b...|http://blogs.reut...| Reuters UK \(blog\)| b|dPhGU51DcrolUIMxb...| blogs.reuters.com|1394470501587|
 13|Europe reaches cr...|http://in.reuters...| Reuters| b|dPhGU51DcrolUIMxb...| in.reuters.com|1394470501755|
 14|ECB FOCUS-Stronge...|http://in.reuters...| Reuters| b|dPhGU51DcrolUIMxb...| in.reuters.com|1394470501948|
 15|EU aims for deal ...|http://main.omano...| Oman Daily Observer| b|dPhGU51DcrolUIMxb...|main.omanobserver.om|1394470502141|
 16|Forex - Pound dro...|http://www.nasdaq...| NASDAQ| b|dPhGU51DcrolUIMxb...| www.nasdaq.com|1394470502316|
 17|Noyer Says Strong...|http://www.sfgate...|San Francisco Chr...| b|dPhGU51DcrolUIMxb...| www.sfgate.com|1394470502543|
 18|EU Week Ahead Mar...|http://blogs.wsj....|Wall Street Journ...| b|dPhGU51DcrolUIMxb...| blogs.wsj.com|1394470502744|
 19|ECB member Noyer ...|http://www.ifamag...| IFA Magazine| b|dPhGU51DcrolUIMxb...| www.ifamagazine.com|1394470502946|
 20|Euro Anxieties Wa...|http://www.busine...| Businessweek| b|dPhGU51DcrolUIMxb...|www.businessweek.com|1394470503148|
+---+--------------------+--------------------+--------------------+--------+--------------------+--------------------+-------------+
only showing top 20 rows

In [10]:
news_data.count()

Out[ 7 ]: 422937

There are 422937 news articles

Select the columns needed for analysis

In [13]:
title_category = news_data.select("TITLE","CATEGORY")

In [14]:
title_category.show()

+--------------------+--------+
 TITLE|CATEGORY|
+--------------------+--------+
Fed official says...| b|
Fed's Charles Plo...| b|
US open: Stocks f...| b|
Fed risks falling...| b|
Fed's Plosser: Na...| b|
Plosser: Fed May ...| b|
Fed's Plosser: Ta...| b|
Fed's Plosser exp...| b|
US jobs growth la...| b|
ECB unlikely to e...| b|
ECB unlikely to e...| b|
EU's half-baked b...| b|
Europe reaches cr...| b|
ECB FOCUS-Stronge...| b|
EU aims for deal ...| b|
Forex - Pound dro...| b|
Noyer Says Strong...| b|
EU Week Ahead Mar...| b|
ECB member Noyer ...| b|
Euro Anxieties Wa...| b|
+--------------------+--------+
only showing top 20 rows

Let's check null values in TITLE and CATEGORY columns

In [16]:
def null_value_count(df):
  null_columns_counts = []
  numRows = df.count()
  for k in df.columns:
    nullRows = df.where(col(k).isNull()).count()
    if(nullRows > 0):
      temp = k,nullRows
      null_columns_counts.append(temp)
  return(null_columns_counts)

In [17]:
null_columns_count_list = null_value_count(title_category)


In [18]:
spark.createDataFrame(null_columns_count_list, ['Column_With_Null_Value', 'Null_Values_Count']).show()

+----------------------+-----------------+
Column_With_Null_Value|Null_Values_Count|
+----------------------+-----------------+
 TITLE| 389|
 CATEGORY| 516|
+----------------------+-----------------+

There are 389 empty titles and 516 categories

Drop the null values

In [21]:
title_category = title_category.dropna()

In [22]:
title_category.count()

Out[ 23 ]: 422421

In [23]:
title_category.show(truncate=False)

+---------------------------------------------------------------------------+--------+
TITLE |CATEGORY|
+---------------------------------------------------------------------------+--------+
Fed official says weak data caused by weather, should not slow taper |b |
Fed's Charles Plosser sees high bar for change in pace of tapering |b |
US open: Stocks fall after Fed official hints at accelerated tapering |b |
Fed risks falling 'behind the curve', Charles Plosser says |b |
Fed's Plosser: Nasty Weather Has Curbed Job Growth |b |
Plosser: Fed May Have to Accelerate Tapering Pace |b |
Fed's Plosser: Taper pace may be too slow |b |
Fed's Plosser expects US unemployment to fall to 6.2% by the end of 2014 |b |
US jobs growth last month hit by weather:Fed President Charles Plosser |b |
ECB unlikely to end sterilisation of SMP purchases - traders |b |
ECB unlikely to end sterilization of SMP purchases: traders |b |
EU's half-baked bank union could work |b |
Europe reaches crunch point on banking union |b |
ECB FOCUS-Stronger euro drowns out ECB's message to keep rates low for ...|b |
EU aims for deal on tackling failing banks |b |
Forex - Pound drops to one-month lows against euro |b |
Noyer Says Strong Euro Creates Unwarranted Economic Pressure |b |
EU Week Ahead March 10-14: Bank Resolution, Transparency, Ukraine |b |
ECB member Noyer is 'very open to all kinds of tools' |b |
Euro Anxieties Wane as Bunds Top Treasuries, Spain Debt Rallies |b |
+---------------------------------------------------------------------------+--------+
only showing top 20 rows

In [24]:
title_category.select("Category").distinct().count()

Out[ 44 ]: 265

Top 20 news categories

In [26]:
title_category.groupBy("Category").count().orderBy(col("count").desc()).show(truncate=False)

+--------------------+------+
Category |count |
+--------------------+------+
e |152127|
b |115935|
t |108237|
m |45616 |
Us Magazine |31 |
Contactmusic.com |20 |
GossipCop |20 |
Complex.com |12 |
CBS News |12 |
The Hollywood Gossip|11 |
HipHopDX |11 |
HeadlinePlanet.com |10 |
We Got This Covered |10 |
Gamepur |8 |
TooFab.com |7 |
Wetpaint |7 |
Consequence of Sound|7 |
WorstPreviews.com |7 |
The Escapist |6 |
NBC Bay Area |5 |
+--------------------+------+
only showing top 20 rows

Top 20 news title

In [28]:
title_category.groupBy("TITLE").count().orderBy(col("count").desc()).show(truncate=False)

+----------------------------------------------------------------------------------+-----+
TITLE |count|
+----------------------------------------------------------------------------------+-----+
The article requested cannot be found! Please refresh your browser or go back ...|145 |
Posted by Parvez Jabri |59 |
Business Highlights |59 |
Posted by Imaduddin |53 |
Posted by Shoaib-ur-Rehman Siddiqui |52 |
(click the phrases to see a list) |51 |
Business Wire |41 |
PR Newswire |38 |
Posted by Muhammad Iqbal |35 |
Change text size for the story |34 |
Get the Most Popular Beauty World News Stories in a Weekly Newsletter |34 |
Business briefs |33 |
International markets roundup |33 |
India Morning Call-Global Markets |27 |
10 Things to Know for Today |22 |
Breaking news |21 |
Perez Recommends |19 |
Texas Weekly Gas Price Update and Outlook |18 |
From ColumbusAlive.com |18 |
The Daily Dish |17 |
+----------------------------------------------------------------------------------+-----+
only showing top 20 rows

#### Let's clean the dataset

Removing numbers from titles

In [31]:
title_category = title_category.withColumn("only_str",regexp_replace(col('TITLE'), '\d+', ''))

In [32]:
title_category.select("TITLE","only_str").show(truncate=False)

+---------------------------------------------------------------------------+---------------------------------------------------------------------------+
TITLE |only_str |
+---------------------------------------------------------------------------+---------------------------------------------------------------------------+
Fed official says weak data caused by weather, should not slow taper |Fed official says weak data caused by weather, should not slow taper |
Fed's Charles Plosser sees high bar for change in pace of tapering |Fed's Charles Plosser sees high bar for change in pace of tapering |
US open: Stocks fall after Fed official hints at accelerated tapering |US open: Stocks fall after Fed official hints at accelerated tapering |
Fed risks falling 'behind the curve', Charles Plosser says |Fed risks falling 'behind the curve', Charles Plosser says |
Fed's Plosser: Nasty Weather Has Curbed Job Growth |Fed's Plosser: Nasty Weather Has Curbed Job Growth |
Plosser: Fed May Have to Accelerate Tapering Pace |Plosser: Fed May Have to Accelerate Tapering Pace |
Fed's Plosser: Taper pace may be too slow |Fed's Plosser: Taper pace may be too slow |
Fed's Plosser expects US unemployment to fall to 6.2% by the end of 2014 |Fed's Plosser expects US unemployment to fall to .% by the end of |
US jobs growth last month hit by weather:Fed President Charles Plosser |US jobs growth last month hit by weather:Fed President Charles Plosser |
ECB unlikely to end sterilisation of SMP purchases - traders |ECB unlikely to end sterilisation of SMP purchases - traders |
ECB unlikely to end sterilization of SMP purchases: traders |ECB unlikely to end sterilization of SMP purchases: traders |
EU's half-baked bank union could work |EU's half-baked bank union could work |
Europe reaches crunch point on banking union |Europe reaches crunch point on banking union |
ECB FOCUS-Stronger euro drowns out ECB's message to keep rates low for ...|ECB FOCUS-Stronger euro drowns out ECB's message to keep rates low for ...|
EU aims for deal on tackling failing banks |EU aims for deal on tackling failing banks |
Forex - Pound drops to one-month lows against euro |Forex - Pound drops to one-month lows against euro |
Noyer Says Strong Euro Creates Unwarranted Economic Pressure |Noyer Says Strong Euro Creates Unwarranted Economic Pressure |
EU Week Ahead March 10-14: Bank Resolution, Transparency, Ukraine |EU Week Ahead March -: Bank Resolution, Transparency, Ukraine |
ECB member Noyer is 'very open to all kinds of tools' |ECB member Noyer is 'very open to all kinds of tools' |
Euro Anxieties Wane as Bunds Top Treasuries, Spain Debt Rallies |Euro Anxieties Wane as Bunds Top Treasuries, Spain Debt Rallies |
+---------------------------------------------------------------------------+---------------------------------------------------------------------------+
only showing top 20 rows

Spliting text into words

In [34]:
regex_tokenizer = RegexTokenizer(inputCol="only_str", outputCol="words", pattern="\\W")
raw_words = regex_tokenizer.transform(title_category)

In [35]:
raw_words.show()

+--------------------+--------+--------------------+--------------------+
 TITLE|CATEGORY| only_str| words|
+--------------------+--------+--------------------+--------------------+
Fed official says...| b|Fed official says...|[fed, official, s...|
Fed's Charles Plo...| b|Fed's Charles Plo...|[fed, s, charles,...|
US open: Stocks f...| b|US open: Stocks f...|[us, open, stocks...|
Fed risks falling...| b|Fed risks falling...|[fed, risks, fall...|
Fed's Plosser: Na...| b|Fed's Plosser: Na...|[fed, s, plosser,...|
Plosser: Fed May ...| b|Plosser: Fed May ...|[plosser, fed, ma...|
Fed's Plosser: Ta...| b|Fed's Plosser: Ta...|[fed, s, plosser,...|
Fed's Plosser exp...| b|Fed's Plosser exp...|[fed, s, plosser,...|
US jobs growth la...| b|US jobs growth la...|[us, jobs, growth...|
ECB unlikely to e...| b|ECB unlikely to e...|[ecb, unlikely, t...|
ECB unlikely to e...| b|ECB unlikely to e...|[ecb, unlikely, t...|
EU's half-baked b...| b|EU's half-baked b...|[eu, s, half, bak...|
Europe reaches cr...| b|Europe reaches cr...|[europe, reaches,...|
ECB FOCUS-Stronge...| b|ECB FOCUS-Stronge...|[ecb, focus, stro...|
EU aims for deal ...| b|EU aims for deal ...|[eu, aims, for, d...|
Forex - Pound dro...| b|Forex - Pound dro...|[forex, pound, dr...|
Noyer Says Strong...| b|Noyer Says Strong...|[noyer, says, str...|
EU Week Ahead Mar...| b|EU Week Ahead Mar...|[eu, week, ahead,...|
ECB member Noyer ...| b|ECB member Noyer ...|[ecb, member, noy...|
Euro Anxieties Wa...| b|Euro Anxieties Wa...|[euro, anxieties,...|
+--------------------+--------+--------------------+--------------------+
only showing top 20 rows

Removing stop words

In [37]:
remover = StopWordsRemover(inputCol="words", outputCol="filtered")
words_df = remover.transform(raw_words)

In [38]:
words_df.select("words","filtered").show(truncate=False)

+-------------------------------------------------------------------------------------+-------------------------------------------------------------------------------+
words |filtered |
+-------------------------------------------------------------------------------------+-------------------------------------------------------------------------------+
[fed, official, says, weak, data, caused, by, weather, should, not, slow, taper] |[fed, official, says, weak, data, caused, weather, slow, taper] |
[fed, s, charles, plosser, sees, high, bar, for, change, in, pace, of, tapering] |[fed, charles, plosser, sees, high, bar, change, pace, tapering] |
[us, open, stocks, fall, after, fed, official, hints, at, accelerated, tapering] |[us, open, stocks, fall, fed, official, hints, accelerated, tapering] |
[fed, risks, falling, behind, the, curve, charles, plosser, says] |[fed, risks, falling, behind, curve, charles, plosser, says] |
[fed, s, plosser, nasty, weather, has, curbed, job, growth] |[fed, plosser, nasty, weather, curbed, job, growth] |
[plosser, fed, may, have, to, accelerate, tapering, pace] |[plosser, fed, may, accelerate, tapering, pace] |
[fed, s, plosser, taper, pace, may, be, too, slow] |[fed, plosser, taper, pace, may, slow] |
[fed, s, plosser, expects, us, unemployment, to, fall, to, by, the, end, of] |[fed, plosser, expects, us, unemployment, fall, end] |
[us, jobs, growth, last, month, hit, by, weather, fed, president, charles, plosser] |[us, jobs, growth, last, month, hit, weather, fed, president, charles, plosser]|
[ecb, unlikely, to, end, sterilisation, of, smp, purchases, traders] |[ecb, unlikely, end, sterilisation, smp, purchases, traders] |
[ecb, unlikely, to, end, sterilization, of, smp, purchases, traders] |[ecb, unlikely, end, sterilization, smp, purchases, traders] |
[eu, s, half, baked, bank, union, could, work] |[eu, half, baked, bank, union, work] |
[europe, reaches, crunch, point, on, banking, union] |[europe, reaches, crunch, point, banking, union] |
[ecb, focus, stronger, euro, drowns, out, ecb, s, message, to, keep, rates, low, for]|[ecb, focus, stronger, euro, drowns, ecb, message, keep, rates, low] |
[eu, aims, for, deal, on, tackling, failing, banks] |[eu, aims, deal, tackling, failing, banks] |
[forex, pound, drops, to, one, month, lows, against, euro] |[forex, pound, drops, one, month, lows, euro] |
[noyer, says, strong, euro, creates, unwarranted, economic, pressure] |[noyer, says, strong, euro, creates, unwarranted, economic, pressure] |
[eu, week, ahead, march, bank, resolution, transparency, ukraine] |[eu, week, ahead, march, bank, resolution, transparency, ukraine] |
[ecb, member, noyer, is, very, open, to, all, kinds, of, tools] |[ecb, member, noyer, open, kinds, tools] |
[euro, anxieties, wane, as, bunds, top, treasuries, spain, debt, rallies] |[euro, anxieties, wane, bunds, top, treasuries, spain, debt, rallies] |
+-------------------------------------------------------------------------------------+-------------------------------------------------------------------------------+
only showing top 20 rows

Lets encode column of category to a column of category indices

In [40]:
indexer = StringIndexer(inputCol="CATEGORY", outputCol="categoryIndex")
feature_data = indexer.fit(words_df).transform(words_df)

In [41]:
feature_data.select("CATEGORY","categoryIndex").show()

+--------+-------------+
CATEGORY|categoryIndex|
+--------+-------------+
 b| 1.0|
 b| 1.0|
 b| 1.0|
 b| 1.0|
 b| 1.0|
 b| 1.0|
 b| 1.0|
 b| 1.0|
 b| 1.0|
 b| 1.0|
 b| 1.0|
 b| 1.0|
 b| 1.0|
 b| 1.0|
 b| 1.0|
 b| 1.0|
 b| 1.0|
 b| 1.0|
 b| 1.0|
 b| 1.0|
+--------+-------------+
only showing top 20 rows

Converting text into vectors of token counts.

In [43]:
cv = CountVectorizer(inputCol="filtered", outputCol="features")
model = cv.fit(words_label_index)
countVectorizer_feateures = model.transform(words_label_index)

Partition Training & Test sets

In [45]:
(trainingData, testData) = countVectorizer_feateures.randomSplit([0.8, 0.2],seed = 11)

#### Model Training and Evaluation

In [47]:
nb = NaiveBayes(modelType="multinomial",labelCol="categoryIndex", featuresCol="features")
nbModel = nb.fit(trainingData)
nb_predictions = nbModel.transform(testData)

In [48]:
nb_predictions.select("prediction", "categoryIndex", "features").show(5)

+----------+-------------+--------------------+
prediction|categoryIndex| features|
+----------+-------------+--------------------+
 0.0| 0.0|(49043,[6,53,162,...|
 0.0| 0.0|(49043,[17,552,10...|
 0.0| 0.0|(49043,[291,552,8...|
 0.0| 0.0|(49043,[20,46,153...|
 0.0| 6.0|(49043,[150,340,4...|
+----------+-------------+--------------------+
only showing top 5 rows

In [49]:
evaluator = MulticlassClassificationEvaluator(labelCol="categoryIndex", predictionCol="prediction", metricName="accuracy")
nb_accuracy = evaluator.evaluate(nb_predictions)
print("Accuracy of NaiveBayes is = %g"% (nb_accuracy))
print("Test Error of NaiveBayes = %g " % (1.0 - nb_accuracy))

Accuracy of NaiveBayes is = 0.926474
Test Error of NaiveBayes = 0.0735258

Reference 

https://spark.apache.org/docs/latest/ml-guide.html